In [1]:
!git clone https://github.com/wandb/edu.git
%cd edu/rag-advanced
!pip install -qqq -r requirements.txt
%cd notebooks

import nltk

nltk.download("wordnet")
nltk.download("punkt")

Cloning into 'edu'...
remote: Enumerating objects: 4614, done.
remote: Counting objects: 100% (1516/1516), done.
remote: Compressing objects: 100% (611/611), done.
remote: Total 4614 (delta 1120), reused 1117 (delta 901), pack-reused 3098 (from 1)
Receiving objects: 100% (4614/4614), 41.87 MiB | 13.87 MiB/s, done.
Resolving deltas: 100% (2515/2515), done.
/content/edu/rag-advanced
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 1.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.4/73.4 kB 5.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 982.3/982.3 kB 24.9 MB/s eta 0:00

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [2]:
import getpass
import os

os.environ["COHERE_API_KEY"] = getpass.getpass("Please enter your COHERE_API_KEY")

Please enter your COHERE_API_KEY··········


In [3]:
import asyncio

import nest_asyncio

nest_asyncio.apply()

import cohere
import weave
from IPython.display import Markdown

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


In [4]:
WANDB_PROJECT = "rag-course"

weave_client = weave.init(WANDB_PROJECT)

Please login to Weights & Biases (https://wandb.ai/) to continue:


wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Logged in as Weights & Biases user: shikhar_dave.
View Weave data at https://wandb.ai/shikhar_dave-iit-jodhpur/rag-course/weave


In [5]:
# Reload the data from Chapter 3
chunked_data = weave.ref(
    "weave:///rag-course/rag-course/object/chunked_data:Ij9KThmiZQ9ljpCm8rVXTJlCaAbY2qC0zX6UJkBWHQ0"
).get()
chunked_data = weave.ref("chunked_data:latest").get()

print(chunked_data.rows[:2])

chunked_data = list(map(dict, chunked_data.rows[:]))

[WeaveDict({'content': '--- description: Log and visualize data without a W&B account displayed_sidebar: default --- # Anonymous Mode Are you publishing code that you want anyone to be able to run easily? Use Anonymous Mode to let someone run your code, see a W&B dashboard, and visualize results without needing to create a W&B account first. Allow results to be logged in Anonymous Mode with `wandb.init(`**`anonymous="allow"`**`)` :::info **Publishing a paper?** Please [cite W&B](https://docs.wandb.ai/company/academics#bibtex-citation), and if you have questions about how to make your code accessible while using W&B, reach out to us at support@wandb.com. ::: ### How does someone without an account see results? If someone runs your script and you have to set `anonymous="allow"`: 1. **Auto-create temporary account:** W&B checks for an account that\'s already signed in. If there\'s no account, we automatically create a new anonymous account and save that API key for the session. 2. **Log r

In [6]:
from scripts.query_enhancer import QueryEnhancer
from scripts.utils import display_source

query_enhancer = QueryEnhancer()

In [7]:
response = await query_enhancer.predict("How do I log images in lightning with wandb?")
response

🍩 https://wandb.ai/shikhar_dave-iit-jodhpur/rag-course/r/call/01928e2b-2bed-7c83-af60-1333cc417cad


{'query': 'How do I log images in lightning with wandb?',
 'language': 'en',
 'search_queries': ['wandb lightning log image',
  'wandb log image',
  'wandb log image lightning',
  'wandb log image in lightning module',
  'log image in wandb lightning'],
 'intents': [{'intent': 'integrations',
   'reason': 'The user is asking about logging images in Lightning, which involves integrating Weights & Biases with another framework.'},
  {'intent': 'product_features',
   'reason': 'The user is seeking guidance on how to perform a specific task of logging images, which relates to using a feature of Weights & Biases.'}]}

In [8]:
from scripts.retriever import BM25Retriever

retriever = BM25Retriever()
retriever.index_data(chunked_data)

DEBUG:bm25s:Building index from IDs objects


In [9]:
from scripts.response_generator import QueryEnhanedResponseGenerator

display_source(QueryEnhanedResponseGenerator)

    1 class QueryEnhanedResponseGenerator(weave.Model):                                                            
    2     """                                                                                                      
    3     A response generator model that enhances queries with additional, language, and intents.                 
    4                                                                                                              
    5     Attributes:                                                                                              
    6         model (str): The model name to be used for generating responses.                                     
    7         prompt (str): The prompt to be used for generating responses.                                        
    8         client (cohere.AsyncClient): The asynchronous Cohere client for interacting with the Cohere API.     
    9     """                                                                                                      
   10                                                                                                              
   11     model: str                                                                                               
   12     prompt: str                                                                                              
   13     client: cohere.AsyncClientV2 = None                                                                      
   14                                                                                                              
   15     def __init__(self, **kwargs):                                                                            
   16         """                                                                                                  
   17         Initialize the QueryEnhanedResponseGenerator with the provided keyword arguments.                    
   18         Sets up the asynchronous Cohere client using the API key from environment variables.                 
   19         """                                                                                                  
   20         super().__init__(**kwargs)                                                                           
   21         self.client = cohere.AsyncClientV2(api_key=os.environ["COHERE_API_KEY"])                             
   22                                                                                                              
   23     @weave.op()                                                                                              
   24     def generate_context(self, context: List[Dict[str, any]]) -> List[Dict[str, any]]:                       
   25         """                                                                                                  
   26         Generate a list of contexts from the provided context list.                                          
   27                                                                                                              
   28         Args:                                                                                                
   29             context (List[Dict[str, any]]): A list of dictionaries containing context data.                  
   30                                                                                                              
   31         Returns:                                                                                             
   32             List[Dict[str, any]]: A list of dictionaries with 'source' and 'text' keys.                      
   33         """                                                                                                  
   34         contexts = [                                                                                         
   35             {"data": {"source": item["source"], "t

In [12]:
from scripts.rag_pipeline import QueryEnhancedRAGPipeline

display_source(QueryEnhancedRAGPipeline)

   1 class QueryEnhancedRAGPipeline(weave.Model):                                                                  
   2     """                                                                                                       
   3     A Query-Enhanced Retrieval-Augmented Generation (RAG) pipeline.                                           
   4                                                                                                               
   5     Attributes:                                                                                               
   6         query_enhancer (weave.Model): The model used for enhancing the input query.                           
   7         retriever (weave.Model): The model used for retrieving relevant documents.                            
   8         response_generator (weave.Model): The model used for generating responses.                            
   9         top_k (int): The number of top documents to retrieve.                                                 
  10     """                                                                                                       
  11                                                                                                               
  12     query_enhancer: weave.Model = None                                                                        
  13     retriever: weave.Model = None                                                                             
  14     response_generator: weave.Model = None                                                                    
  15     top_k: int = 5                                                                                            
  16                                                                                                               
  17     @weave.op()                                                                                               
  18     async def predict(self, query: str):                                                                      
  19         """                                                                                                   
  20         Predicts a response based on the enhanced input query.                                                
  21                                                                                                               
  22         Args:                                                                                                 
  23             query (str): The input query string.                                                              
  24                                                                                                               
  25         Returns:                                                                                              
  26             The generated response based on the retrieved context, language, and intent actions.              
  27         """                                                                                                   
  28         # enhance the query                                                                                   
  29         enhanced_query = await self.query_enhancer.predict(query)                                             
  30         user_query = enhanced_query["query"]                                                                  
  31                                                                                                               
  32         avoid_intents = [                                                                                     
  33             "unrelated",                                                                                      
  34             "needs_more_info",                                                                                
  35             "opinion_request",                     

In [10]:
QUERY_ENHANCED_PROMPT = open("prompts/query_enhanced_system.txt").read()

response_generator = QueryEnhanedResponseGenerator(
    model="command-r", prompt=QUERY_ENHANCED_PROMPT, client=cohere.AsyncClientV2()
)

In [13]:
query_enhanced_rag_pipeline = QueryEnhancedRAGPipeline(
    query_enhancer=query_enhancer,
    retriever=retriever,
    response_generator=response_generator,
    top_k=2,
)

response = await query_enhanced_rag_pipeline.predict(
    "How do I log images in lightning with wandb?"
)


Markdown(response)

🍩 https://wandb.ai/shikhar_dave-iit-jodhpur/rag-course/r/call/01928e36-1d7f-7ea2-bf0e-6db816236b46


You can use WandbLogger to log images in lightning. Below is an example of the code you can use:
```
import torch
import wandb
import lightning.pytorch as pl
from lightning.pytorch.loggers import WandbLogger

wandb_logger = WandbLogger()
trainer = pl.Trainer(logger=wandb_logger)
```

You can also log images using this method:
```
import torch import wandb import lightning.pytorch as pl from lightning.pytorch.loggers import WandbLogger # or from wandb.integration.lightning.fabric import WandbLogger 

class LogPredictionSamplesCallback(Callback):
def on_validation_batch_end(self, trainer, pl_module, outputs, batch, batch_idx, dataloader_idx):
        """Called when the validation batch ends."""

        # `outputs` comes from `LightningModule.validation_step` # which corresponds to our model predictions in this case

        # Let's log 20 sample image predictions from the first batch
        if batch_idx == 0:
            n = 20
            x, y = batch
            images = [img for img in x[:n]]
            captions = [f"Ground Truth: {y_i} - Prediction: {y_pred}" for y_i, y_pred in zip(y[:n], outputs[:n])]

            # Option 1: log images with `WandbLogger.log_image`
            wandb_logger.log_image(key="sample_images", images=images, caption=captions)

            # Option 2: log images and predictions as a W&B Table
            columns = ["image", "ground truth", "prediction"]
            data = [
                 [wandb.Image(x_i), y_i, y_pred]
                for x_i, y_i, y_pred in list(zip(x[:n], y[:n], outputs[:n]))
            ]
            wandb_logger.log_table(key="sample_table", columns=columns, data=data)

        trainer = pl.Trainer(callbacks=[LogPredictionSamplesCallback()])
```

You can also log images using the `wandb.Image()` function. The function converts numpy arrays or instances of `PILImage` to PNGs by default.

In [14]:
eval_dataset = weave.ref(
    "weave:///rag-course/dev/object/Dataset:Qj4IFICc2EbdXu5A5UuhkPiWgxM1GvJMIvXEyv1DYnM"
).get()

print(eval_dataset.rows[:2])

[WeaveDict({'question': 'How can I access the run object from the Lightning WandBLogger function?', 'answer': "In PyTorch Lightning, the `WandbLogger` is used to log metrics, model weights, and other data to Weights & Biases during training. To access the `wandb.Run` object from within a `LightningModule` when using `WandbLogger`, you can use the `Trainer.logger.experiment` attribute. This attribute provides direct access to the underlying `wandb.Run` object, allowing you to interact with the Weights & Biases API directly.\n\nHere's how you can access the `wandb.Run` object using `WandbLogger` in PyTorch Lightning:\n\n```python\nfrom pytorch_lightning import Trainer, LightningModule\nfrom pytorch_lightning.loggers import WandbLogger\n\nclass MyModel(LightningModule):\n    def training_step(self, batch, batch_idx):\n        # Your training logic here\n        loss = ...\n\n        # Log metrics\n        self.log('train_loss', loss)\n\n        # Access the wandb.Run object\n        run =

In [15]:
from scripts.rag_pipeline import SimpleRAGPipeline
from scripts.response_generator import SimpleResponseGenerator

INITIAL_PROMPT = open("prompts/initial_system.txt", "r").read()
response_generator = SimpleResponseGenerator(model="command-r", prompt=INITIAL_PROMPT)
simple_rag_pipeline = SimpleRAGPipeline(
    retriever=retriever, response_generator=response_generator, top_k=5
)